# Customer Behavior Analysis

In [3]:
!pip install pandas sqlalchemy mysql-connector-python

In [4]:
import pandas as pd
import sqlalchemy
import mysql.connector

print("Libraries installed successfully!")

Libraries installed successfully!


In [7]:
# Connect to MySQL Server (without specifying a database)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Akshaya@22"  
)

cursor = conn.cursor()

# Create Database
cursor.execute("CREATE DATABASE IF NOT EXISTS customer_db;")
print("Database 'customer_db' created successfully!")

Database 'customer_db' created successfully!


In [9]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse

# Database connection details
db_user = "root"  
db_password = urllib.parse.quote_plus("Akshaya@22")  
db_host = "localhost"
db_name = "customer_db"

# Create SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")

# Define CSV file paths
file_paths = {
    "customer_journey": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\customer_journey.csv",
    "customer_reviews": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\customer_reviews.csv",
    "customers": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\customers.csv",
    "engagement_data": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\engagement_data.csv",
    "geography": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\geography.csv",
    "products": r"C:\Users\Aksha\OneDrive\Documents\Akshaya\GUVI\products.csv",
}

# Load and insert each file
for table_name, file_path in file_paths.items():
    try:
        df = pd.read_csv(file_path)
        df.to_sql(table_name, con=engine, if_exists="replace", index=False)
        print(f"Data inserted successfully into {table_name}")
    except Exception as e:
        print(f"Error loading {table_name}: {e}")


Data inserted successfully into customer_journey
Data inserted successfully into customer_reviews
Data inserted successfully into customers
Data inserted successfully into engagement_data
Data inserted successfully into geography
Data inserted successfully into products


# Marketing Manager

In [9]:
# Query to analyze customer engagement factors
query = """
SELECT 
    c.CustomerID, 
    c.Age, 
    g.Country, 
    g.City, 
    e.ViewsClicksCombined, 
    e.Likes, 
    r.Rating, 
    LENGTH(r.ReviewText) AS ReviewLength, 
    j.Stage, 
    j.Duration 
FROM customers c
JOIN customer_journey j ON c.CustomerID = j.CustomerID
JOIN customer_reviews r ON c.CustomerID = r.CustomerID
JOIN geography g ON c.GeographyID = g.GeographyID
JOIN engagement_data e ON j.ProductID = e.ProductID
ORDER BY e.ViewsClicksCombined DESC, e.Likes DESC;
"""

# Execute query
df = pd.read_sql(query, con=engine)

# Show top results
print(df.head())

   CustomerID  Age  Country    City ViewsClicksCombined  Likes  Rating  \
0          93   41    Spain  Madrid           9759-3095    400       5   
1          93   41    Spain  Madrid           9759-3095    400       3   
2           9   29  Austria  Vienna           9759-3095    400       3   
3           9   29  Austria  Vienna           9759-3095    400       3   
4          93   41    Spain  Madrid           9759-3095    400       4   

   ReviewLength        Stage  Duration  
0            34  ProductPage     279.0  
1            42  ProductPage     279.0  
2            23  ProductPage     292.0  
3            23     Homepage     187.0  
4            27  ProductPage     279.0  


In [11]:
# SQL query to identify drop-off stages
query = """
SELECT 
    UPPER (Stage) AS DropoffStage, 
    COUNT(DISTINCT CustomerID) AS DropoffCount
FROM customer_journey
WHERE VisitDate = (
    SELECT MAX(VisitDate) FROM customer_journey AS sub 
    WHERE sub.CustomerID = customer_journey.CustomerID
)
GROUP BY DropoffStage
ORDER BY DropoffCount DESC;
"""
dropoff_data = pd.read_sql(query, con=engine)

# Display results
print(dropoff_data)

  DropoffStage  DropoffCount
0     HOMEPAGE            32
1  PRODUCTPAGE            20
2     CHECKOUT            13


In [13]:
# Query: Top Performing Products
query_products = """
SELECT 
    p.ProductName, 
    COUNT(cj.CustomerID) AS PurchaseCount
FROM customer_journey cj
JOIN products p ON cj.ProductID = p.ProductID
WHERE cj.Action = 'Purchase'  -- Customers who completed a purchase
GROUP BY p.ProductName
ORDER BY PurchaseCount DESC
LIMIT 10;
"""

# Query: Top Performing Locations
query_locations = """
SELECT 
    g.Country, g.City, 
    COUNT(c.CustomerID) AS CustomerCount
FROM customers c
JOIN geography g ON c.GeographyID = g.GeographyID
GROUP BY g.Country, g.City
ORDER BY CustomerCount DESC
LIMIT 10;
"""

# Query: Top Performing Customer Segments (by Age Group)
query_segments = """
SELECT 
    CASE 
        WHEN Age BETWEEN 18 AND 25 THEN '18-25'
        WHEN Age BETWEEN 26 AND 35 THEN '26-35'
        WHEN Age BETWEEN 36 AND 45 THEN '36-45'
        WHEN Age BETWEEN 46 AND 60 THEN '46-60'
        ELSE '60+' 
    END AS AgeGroup,
    COUNT(CustomerID) AS CustomerCount
FROM customers
GROUP BY AgeGroup
ORDER BY CustomerCount DESC;
"""

# Execute Queries
top_products = pd.read_sql(query_products, con=engine)
top_locations = pd.read_sql(query_locations, con=engine)
top_segments = pd.read_sql(query_segments, con=engine)

# Display Results
print(" Top Performing Products:\n", top_products)
print("\n Top Performing Locations:\n", top_locations)
print("\n Top Performing Customer Segments:\n", top_segments)


 Top Performing Products:
        ProductName  PurchaseCount
0    Climbing Rope              2
1        Surfboard              2
2  Fitness Tracker              1
3            Kayak              1

 Top Performing Locations:
        Country       City  CustomerCount
0        Spain     Madrid             18
1        Italy       Rome             12
2      Germany     Berlin             11
3      Austria     Vienna             10
4           UK     London             10
5  Netherlands  Amsterdam              9
6      Belgium   Brussels              9
7       Sweden  Stockholm              8
8  Switzerland     Zurich              8
9       France      Paris              5

 Top Performing Customer Segments:
   AgeGroup  CustomerCount
0    36-45             25
1    46-60             24
2    26-35             22
3    18-25             15
4      60+             14


In [15]:
# Analyze impact of customer reviews on purchases
query = """
SELECT 
    p.ProductName, 
    r.Rating, 
    AVG(LENGTH(r.ReviewText)) AS AvgReviewLength, 
    COUNT(DISTINCT cj.CustomerID) AS PurchaseCount
FROM customer_reviews r
LEFT JOIN customer_journey cj ON r.ProductID = cj.ProductID  -- Removed CustomerID match
LEFT JOIN products p ON r.ProductID = p.ProductID
WHERE  cj.Stage = 'Checkout' 
GROUP BY p.ProductName, r.Rating
ORDER BY PurchaseCount DESC;
"""

# Execute SQL query
review_impact = pd.read_sql(query, con=engine)

# Display results
print(" Impact of Reviews on Purchasing Behavior:\n", review_impact)


 Impact of Reviews on Purchasing Behavior:
         ProductName  Rating  AvgReviewLength  PurchaseCount
0     Boxing Gloves       4          48.0000              3
1         Ski Boots       3          40.0000              3
2         Ski Boots       4          37.2500              3
3         Ski Boots       5          34.0000              3
4         Surfboard       2          39.0000              3
5         Surfboard       3          32.0000              3
6         Surfboard       4          45.3333              3
7         Surfboard       5          38.0000              3
8     Climbing Rope       2          30.0000              2
9     Climbing Rope       3          40.0000              2
10    Climbing Rope       4          29.0000              2
11    Climbing Rope       5          36.6667              2
12       Volleyball       3          39.0000              2
13       Volleyball       4          41.0000              2
14       Volleyball       5          39.0000            

In [17]:
# SQL Query to Compare Repeat vs. First-Time Buyers
query = """
WITH PurchaseCounts AS (
    SELECT CustomerID, COUNT(*) AS PurchaseCount
    FROM customer_journey
    WHERE Action = 'Purchase'  -- Considering only completed purchases
    GROUP BY CustomerID
)
SELECT 
    CASE 
        WHEN PurchaseCount = 1 THEN 'First-Time Buyer'
        ELSE 'Repeat Buyer'
    END AS BuyerType,
    COUNT(CustomerID) AS BuyerCount
FROM PurchaseCounts
GROUP BY BuyerType;
"""

# Execute SQL query
buyer_comparison = pd.read_sql(query, con=engine)

# Display results
print(" Repeat vs. First-Time Buyers:\n", buyer_comparison)

 Repeat vs. First-Time Buyers:
           BuyerType  BuyerCount
0  First-Time Buyer           6


# Customer Experience Manager

In [19]:
# SQL Query to Find Complaints with Product Details
query = """
SELECT 
    r.ProductID, 
    p.ProductName, 
    r.ReviewText, 
    r.Rating
FROM customer_reviews r
JOIN products p ON r.ProductID = p.ProductID
WHERE r.Rating <= 2
ORDER BY r.Rating ASC
LIMIT 10;
"""

# Execute the query
complaints_df = pd.read_sql(query, con=engine)

# Display the results
print(" Key Customer Complaints by Product:\n", complaints_df)


 Key Customer Complaints by Product:
    ProductID      ProductName  \
0          3         Yoga Mat   
1          4        Dumbbells   
2          6    Tennis Racket   
3          2  Fitness Tracker   
4          7       Basketball   
5         10       Golf Clubs   
6         15    Climbing Rope   
7         16            Kayak   
8         17        Surfboard   

                                          ReviewText  Rating  
0                            Not  worth  the  money.       1  
1   The  product  stopped  working  after  a  month.       1  
2   I  had  a  bad  experience  with  this  product.       1  
3       Product  did    not  meet  my  expectations.       2  
4  The  product    is  okay,  but  the  instructi...       2  
5              Disappointed  with  the  performance.       2  
6                     The  product  arrived    late.       2  
7            Average  experience,  nothing  special.       2  
8            Average  experience,  nothing  special.       2  


In [21]:
#  SQL Query: Find Negative Review Patterns and Product Performance
query = """
WITH NegativeReviews AS (
    SELECT ProductID, COUNT(*) AS NegativeReviewCount, AVG(Rating) AS AvgRating
    FROM customer_reviews
    WHERE Rating <= 2
    GROUP BY ProductID
),
ProductSales AS (
    SELECT ProductID, COUNT(DISTINCT CustomerID) AS PurchaseCount
    FROM customer_journey
    WHERE Action = 'Purchase'
    GROUP BY ProductID
)
SELECT 
    p.ProductName, 
    nr.NegativeReviewCount, 
    nr.AvgRating, 
    ps.PurchaseCount
FROM NegativeReviews nr
JOIN ProductSales ps ON nr.ProductID = ps.ProductID
JOIN products p ON nr.ProductID = p.ProductID
ORDER BY nr.NegativeReviewCount DESC;
"""

# Execute SQL query
negative_review_impact = pd.read_sql(query, con=engine)

# Display results
print(" Pattern Between Negative Reviews and Product Performance:\n", negative_review_impact)


 Pattern Between Negative Reviews and Product Performance:
        ProductName  NegativeReviewCount  AvgRating  PurchaseCount
0  Fitness Tracker                    1        2.0              1
1    Climbing Rope                    1        2.0              2
2            Kayak                    1        2.0              1
3        Surfboard                    1        2.0              2


In [33]:
# SQL Query: Find Highest & Lowest Rated Products
# Query for highest-rated products
query_highest = """
SELECT 
    p.ProductName, 
    AVG(r.Rating) AS AvgRating, 
    COUNT(r.ReviewID) AS ReviewCount
FROM customer_reviews r
JOIN products p ON r.ProductID = p.ProductID
GROUP BY p.ProductName
HAVING ReviewCount > 5  -- Ensure products have enough reviews
ORDER BY AvgRating DESC
LIMIT 5;  -- Get top 5 highest-rated products
"""

# Execute the first query safely
conn = engine.connect()
highest_rated = pd.read_sql(query_highest, con=conn)

print("\nTop 5 Highest-Rated Products:\n", highest_rated)

# Query for lowest-rated products
query_lowest = """
SELECT 
    p.ProductName, 
    AVG(r.Rating) AS AvgRating, 
    COUNT(r.ReviewID) AS ReviewCount
FROM customer_reviews r
JOIN products p ON r.ProductID = p.ProductID
GROUP BY p.ProductName
HAVING ReviewCount > 5  -- Ensure products have enough reviews
ORDER BY AvgRating ASC
LIMIT 5;  -- Get top 5 lowest-rated products
"""

# Execute the second query safely
conn = engine.connect()
lowest_rated = pd.read_sql(query_lowest, con=conn)

print("\nTop 5 Lowest-Rated Products:\n", lowest_rated)


Top 5 Highest-Rated Products:
      ProductName  AvgRating  ReviewCount
0  Climbing Rope     4.0000            6
1      Ski Boots     4.0000            6
2      Surfboard     3.8889            9
3  Tennis Racket     3.8571            7
4       Yoga Mat     3.7143            7

Top 5 Lowest-Rated Products:
        ProductName  AvgRating  ReviewCount
0            Kayak     3.4000           10
1  Fitness Tracker     3.5714            7
2     Swim Goggles     3.6667            9
3         Yoga Mat     3.7143            7
4    Tennis Racket     3.8571            7


In [17]:
#  SQL Query: Find Common Actions Before Conversion
query = """
WITH ConversionCustomers AS (
    SELECT DISTINCT CustomerID
    FROM customer_journey
    WHERE Action IN ('Purchase')  -- Successful conversions
)
SELECT 
    cj.Action, 
    COUNT(DISTINCT cj.CustomerID) AS ActionCount
FROM customer_journey cj
JOIN ConversionCustomers cc ON cj.CustomerID = cc.CustomerID
WHERE cj.Action NOT IN ('Purchase')  -- Exclude final purchase stage
GROUP BY cj.Action
ORDER BY ActionCount DESC
"""

# Execute SQL query
conn = engine.connect()
conversion_actions = pd.read_sql(query, conn)

# Display results
print(" Common Actions Leading to Conversions:\n", conversion_actions)



 Common Actions Leading to Conversions:
      Action  ActionCount
0      View            2
1  Drop-off            1


In [45]:
#  Sentiment Analysis
query = "SELECT ReviewText, Rating FROM customer_reviews"
conn = engine.connect()
df = pd.read_sql(query, conn)

# Assign Sentiment Based on Rating
def assign_sentiment(rating):
    if rating >= 4:
        return "Positive"
    elif rating == 3:
        return "Neutral"
    else:
        return "Negative"

df["Sentiment"] = df["Rating"].apply(assign_sentiment)

# Store the Sentiment Data Back into SQL
df.to_sql("customer_reviews_sentiment", conn, if_exists="replace", index=False)

# Get Sentiment Summary from SQL
query_sentiment = """
SELECT Sentiment, COUNT(*) AS ReviewCount
FROM customer_reviews_sentiment
GROUP BY Sentiment;
"""
sentiment_summary = pd.read_sql(query_sentiment,conn)

# Display Sentiment Analysis Results
print("\n Sentiment Summary Based on Ratings:")
print(sentiment_summary)


 Sentiment Summary Based on Ratings:
  Sentiment  ReviewCount
0   Neutral           29
1  Positive           62
2  Negative            9


In [ ]:
# Close the connection
conn.close()